# This is the new portion of the code that has been inspired by the origin code given


In [3]:


using Plots, Ipopt, JuMP, QuadGK


#JuMP model, Ipopt solver

sys = Model(optimizer_with_attributes(Ipopt.Optimizer,"print_level"=> 5))
set_optimizer_attribute(sys,"tol",1e-8)
set_optimizer_attribute(sys,"constr_viol_tol",1e-6)
set_optimizer_attribute(sys,"max_iter",1000)



# Parameters

# Coefficients du modèlprintln("Valeurs optimales de s(T) : ", [value(s[T])] )
# println("Valeurs optimales de i(T) : ", [value(i[T])] )
# println("Valeurs optimales de m(T) : ", [value(m[T])] )
const A = 10.0
const B = 1.0
const C = 3.0
const T = 20

# Valeurs initiales
const s0 = 0.95
const i0 = 0.05
const m0 = 0.0

# Autres paramètres
const N = 200                   # la valeur N=200 nous donne j1=26.23 
const ε = 1e-9
const dt = T/N

# Valeurs des paramètres SIMR
const b = 0.01
const c = 1.1
const a1 = 0.08
const a2 = 0.005
const g1 = 0.02
const g2 = 0.5
const η = 0.8

# Valeurs maximales des contrôles
const umax = 0.8
const vmax = 0.8


# Variables d'état s(t), i(t), et m(t) et de control u(t) et v(t)

@variables(sys, begin
    0.0 <= Δt                      # time step
    0 <= s[1:N+1]                  # s state (voire s[t=0:N] >= 0 je ne sais pas ce que ça change)
    0 <= i[1:N+1]                  # i state (i[t=0:N] >= 0)
    0 <= m[1:N+1]                  # m state (m[t=0:N] >= 0)
    0 <= u[1:N+1] <= umax         # u control
    0 <= v[1:N+1] <= vmax         # v control
end)




# Expression pour J1(u, v)

@expression(sys, J1, 0.5 * Δt * sum((A * i[t] + B * u[t] + C * v[t] + A * i[t+1] + B * u[t+1] + C * v[t+1]) for t in 1:N))
# Objective
@objective(sys, Min, J1)


# Boundary constraints 

@constraints(sys, begin
    con_s0, s[1] == s0
    con_i0, i[1] == i0
    con_m0, m[1] == m0
    con_iT, i[T] <= 5e-4
end)


# Dynamics
@NLexpressions(sys, begin
    # s' = b - b.s - c.s.i + (a1.i + a2.m).s - η.u.s
    ds[j = 1 : N + 1], ( b - b * s[j] - c * s[j] * i[j] + ( a1 * i[j] + a2 * m[j] ) * s[j] - η * u[j] * s[j] )
    # i' = c.s.i - b.i - (g1 + a1).i + (a1.i + a2.m).i - i.v
    di[j = 1 : N + 1], ( c * s[j] * i[j] - b * i[j] - ( g1 + a1 ) * i[j] + ( a1 * i[j] + a2 *m[j] ) * i[j] - i[j] * v[j])
    # m' = -(a2 + g2 + b).m + (a1.i + a2.m).m + i.v
    dm[j = 1 : N + 1], ( - ( a2 + g2 + b ) * m[j] + ( a1 * i[j] + a2 * m[j] ) * m[j] + i[j] * v[j] )
end)

# Crank-Nicolson scheme
@NLconstraints(sys, begin
    con_ds[j = 1:N ], s[j+1] == s[j] + Δt * (ds[j] + ds[j+1])/2.0
    con_di[j = 1:N ], i[j+1] == i[j] + Δt * (di[j] + di[j+1])/2.0
    con_dm[j = 1:N ], m[j+1] == m[j] + Δt * (dm[j] + dm[j+1])/2.0
end)

(NonlinearConstraintRef{ScalarShape}[(s[2] - (s[1] + (Δt * (subexpression[1] + subexpression[2])) / 2.0)) - 0.0 = 0, (s[3] - (s[2] + (Δt * (subexpression[2] + subexpression[3])) / 2.0)) - 0.0 = 0, (s[4] - (s[3] + (Δt * (subexpression[3] + subexpression[4])) / 2.0)) - 0.0 = 0, (s[5] - (s[4] + (Δt * (subexpression[4] + subexpression[5])) / 2.0)) - 0.0 = 0, (s[6] - (s[5] + (Δt * (subexpression[5] + subexpression[6])) / 2.0)) - 0.0 = 0, (s[7] - (s[6] + (Δt * (subexpression[6] + subexpression[7])) / 2.0)) - 0.0 = 0, (s[8] - (s[7] + (Δt * (subexpression[7] + subexpression[8])) / 2.0)) - 0.0 = 0, (s[9] - (s[8] + (Δt * (subexpression[8] + subexpression[9])) / 2.0)) - 0.0 = 0, (s[10] - (s[9] + (Δt * (subexpression[9] + subexpression[10])) / 2.0)) - 0.0 = 0, (s[11] - (s[10] + (Δt * (subexpression[10] + subexpression[11])) / 2.0)) - 0.0 = 0  …  (s[192] - (s[191] + (Δt * (subexpression[191] + subexpression[192])) / 2.0)) - 0.0 = 0, (s[193] - (s[192] + (Δt * (subexpression[192] + subexpression[193]

In [4]:
# Solves for the control and state
println("Solving...")
status = optimize!(sys)

if termination_status(sys) == MOI.OPTIMAL
    println("Solution is optimal")

elseif termination_status(sys) == MOI.LOCALLY_SOLVED
    println("Local solution found")
elseif termination_status(sys) == MOI.TIME_LIMIT && has_values(sys)
    println("Solution is suboptimal due to a time limit, but a primal solution is available")
else
    error("The model was not solved correctly.")
end
println("Objective value = ", objective_value(sys), "\n")

# Retrieves values (including duals - sign convention according to Pontrjagin max principle)
ss = value.(s)
ii = value.(i)
mm = value.(m)
uu = value.(u)
vv = value.(v)
Δtt = value.(Δt)
println("Δt = ", Δtt)
p_s = -[ dual(con_ds[j]) for j = 1:N - 1]
p_i = -[ dual(con_di[j]) for j = 1:N - 1]
p_m = -[ dual(con_dm[j]) for j = 1:N - 1];

Solving...


This is Ipopt version 3.14.13, running with linear solver MUMPS 5.6.1.

Number of nonzeros in equality constraint Jacobian...:     5003
Number of nonzeros in inequality constraint Jacobian.:        1
Number of nonzeros in Lagrangian Hessian.............:    24203

Total number of variables............................:     1006
                     variables with only lower bounds:      604
                variables with lower and upper bounds:      402
                     variables with only upper bounds:        0
Total number of equality constraints.................:      603
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.6399946e-01 9.40e-01 5.10e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

Objective value = 26.234092980320323



Δt = 0.3695981549798539


In [ ]:
# Plots: state and control
t = (0:N)*dt
s_plot = plot(t, ss, xlabel = "t", ylabel = "s(t)", legend = false, fmt = :png)
i_plot = plot(t, ii, xlabel = "t", ylabel = "i(t)", legend = false, fmt = :png)
m_plot = plot(t, mm, xlabel = "t", ylabel = "m(t)", legend = false, fmt = :png)
u_plot = plot(t, uu, xlabel = "t", ylabel = "u(t)", legend = false, fmt = :png)
v_plot = plot(t, vv, xlabel = "t", ylabel = "v(t)", legend = false, fmt = :png)

display(plot(s_plot, i_plot, m_plot, u_plot, v_plot, layout = (3,2)))

In [ ]:
# Plots: adjoint state
tt = (t[2:end-1] + t[3:end])/2.0
p_s_plot = plot(tt, p_s, xlabel = "t", ylabel = "ps(t)", legend = false, fmt = :png)
p_i_plot = plot(tt, p_i, xlabel = "t", ylabel = "pi(t)", legend = false, fmt = :png)
p_m_plot = plot(tt, p_m, xlabel = "t", ylabel = "pm(t)", legend = false, fmt = :png)
# calculer la taille de pi
display(plot(p_s_plot, p_i_plot, p_m_plot, layout = (2,2)))

In [ ]:
# Afficher les résultats
println("Valeur optimale de J1(u, v) : ", objective_value(sys))
println("Valeur optimale de s(T) : ", [value(s[T])] )
println("Valeur optimale de i(T) : ", [value(i[T])] )
println("Valeur optimale de m(T) : ", [value(m[T])] )

println("Valeur optimale de ps(0) : ", [value(p_s[1])] )
println("Valeur optimale de pi(0) : ", [value(p_i[1])] )
println("Valeur optimale de pm(0) : ", [value(p_m[1])] )
println("Valeur optimale de ps(T) : ", [value(p_s[T])] )
println("Valeur optimale de pi(T) : ", [value(p_i[T])] )
println("Valeur optimale de pm(T) : ", [value(p_m[T])] )
